## it is a file that is used to download all the data used in the project

In [1]:
import csv
import math
import json
import os
import time
import requests
import numpy as np
import pandas as pd 
from datetime import datetime

## Convert date to time stamp

In [37]:
from datetime import datetime
# current date and time
now = datetime.now()
now  = datetime(2018,12,1,0,0,0)
timestamp = datetime.timestamp(now)
print("Time stamp for 1st April 2019 :", timestamp)

Time stamp for 1st April 2019 : 1543602600.0


## Weather data collection

In [22]:
csv_columns = ['time', 'summary', 'icon', 'precipIntensity','precipProbability', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'ozone']
import pandas as pd
from datetime import datetime
#csv_file = "Names.csv"
df = pd.DataFrame(columns=csv_columns)
file = 'Dec_w_2018.csv'
dec = df.to_csv(file,index = False)       
tim = 1543602600
count = 0
debug=0
l = []
while(count<30):
    ##use you api key extracted from darksky
    url = DarkSkyUrl+str(tim)
    response = requests.get(url)
    data = response.text
    parsed = json.loads(data)
    h_data = parsed['hourly']['data']
    for d in h_data:
        d_c = []
        l.append(len(d))
        #if debug==282:
            #print("282 rows data :",len(d))
       # if debug==283:
            #print("**************************************************")
            #print("566 row data:",d)
            #print(len(d))
            #break
        for v in d.keys():
            if v=='time':
                    timestamp = d[v]
                    #print(timestamp)
                    d_c.append(datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))
                    continue
           # else:
                #print(v)
            if v=='precipType':
                continue
            d_c.append(d[v])#(datetime.datetime.fromtimestamp(d[v]).strftime('%Y-%m-%d %H:%M:%S'))
    
        with open(file, "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(d_c)
        fp.close()
        debug = debug+1
    
    tim = tim+24*60*60
    count = count+1

In [8]:
data = pd.read_csv('May_w_2018.csv')
data.head()

,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone
0,2019-08-01 00:00:00,Heavy Rain and Humid,rain,0.4438,0.72,77.61,79.95,77.61,1.0,1002.5,10.59,34.01,237,1.0,0,1.352,271.7
1,2019-08-01 01:00:00,Heavy Rain and Humid,rain,0.3955,0.71,76.91,79.08,76.91,1.0,1002.2,10.11,33.47,253,1.0,0,2.881,273.0
2,2019-08-01 02:00:00,Rain and Humid,rain,0.3184,0.68,77.12,79.32,77.12,1.0,1001.8,9.53,32.46,267,1.0,0,4.549,274.5
3,2019-08-01 03:00:00,Rain and Humid,rain,0.2271,0.64,77.59,79.80,77.59,1.0,1001.5,8.95,30.97,214,1.0,0,5.470,275.3
4,2019-08-01 04:00:00,Rain and Humid,rain,0.1273,0.61,77.54,79.74,77.54,1.0,1001.4,8.33,28.92,236,1.0,0,5.698,275.6


## Solar data collection

In [38]:
def time_to_epoch(yyyy, mm, dd, t):
    t_stamp = str(yyyy).zfill(4) + '-' + str(mm).zfill(2) + '-' + str(dd).zfill(2) + 'T' + str(t)
    time_tuple = time.strptime(t_stamp, '%Y-%m-%dT%H:%M')
    time_epoch = time.mktime(time_tuple)
    return time_epoch


def epoch_to_timestring(n):
    return time.strftime('%Y-%m-%dT%H:%M', time.localtime(n))


# some global variables
private_server_url = 'http://127.0.0.1:5000'
header = {'Content-Type':'application/json'}
imars_base_url = 'http://14.215.130.185:20000'
time_stamps_list = ['UID','00:00', '00:20', '00:40', '01:00', '01:20', '01:40', '02:00', '02:20', '02:40', '03:00', '03:20',
                    '03:40', '04:00', '04:20', '04:40', '05:00', '05:20', '05:40', '06:00', '06:20', '06:40', '07:00',
                    '07:20', '07:40', '08:00', '08:20', '08:40', '09:00', '09:20', '09:40', '10:00', '10:20', '10:40',
                    '11:00', '11:20', '11:40', '12:00', '12:20', '12:40', '13:00', '13:20', '13:40', '14:00', '14:20',
                    '14:40', '15:00', '15:20', '15:40', '16:00', '16:20', '16:40', '17:00', '17:20', '17:40', '18:00',
                    '18:20', '18:40', '19:00', '19:20', '19:40', '20:00', '20:20', '20:40', '21:00', '21:20', '21:40',
                    '22:00', '22:20', '22:40', '23:00', '23:20', '23:40']
device_ids = [33140, 32977, 32684, 32934, 32804, 33096, 32984, 32985, 32725, 19380, 33131]

# Session preserves cookies from login accross https calls
s = requests.session()
# login
auth_data_imars = [('username', ''),('password', '')]  #provide usename and password, can take from IITGn admin
login_response_imars = s.request('POST', imars_base_url + '/login.action', data=auth_data_imars)


# yyyy = 2018
# mm = 10
# dd = 1
# device_id = device_ids[1]

# Func to retreive data from solar panel energy portal for given date
# date format: 'yyyy-mm-dd'
def get_data(device_id, yyyy, mm, dd):
    date = str(yyyy).zfill(4) + '-' + str(mm).zfill(2) + '-' + str(dd).zfill(2)
    params_report = (('level', '3'), ('type', '1'), ('searchIds', device_id), ('searchDate', date))
    # searchIds = ID for panels/inverter...level=?, type=?
    date_report = s.request('GET', imars_base_url + '/reportjson/exportReport.action', params=params_report)
    #print(date_report.url)

    try:
        output = open('./data/{}/{}'.format(device_id, date+'.xls'), 'wb')
    except FileNotFoundError:
        os.makedirs('./data/{}'.format(device_id))
        output = open('./data/{}/{}'.format(device_id, date+'.xls'), 'wb')

    output.write(date_report.content)
    output.close()

    pd_data = pd.ExcelFile('./data/{}/{}'.format(device_id, date+'.xls'))
    pd_data_day = pd.read_excel(pd_data, 'dayReport')
   # print(pd_data_day)
    energy = np.array(pd_data_day[1:2])
    device_name = energy[0][0]
    #print('device_name:',device_name)

    t = np.array(pd_data_day[:1])
    t = np.delete(t, np.s_[:1], 1)
    t_list = list(t[0])

    energy = np.delete(energy, np.s_[:1], 1)
    energy = energy.astype(np.float32, copy=False)
    energy_list = list(energy[0])

    dict_t_e = {}
    # dict_forDF = {'date': date, 'name': device_name}
    for t, e in zip(t_list, energy_list):
        dict_t_e.update({t: e})
    return dict_t_e


yyyy = 2018
mm = 10
#dd = 23
import csv
import datetime

csv_columns =   ['00:00', '00:20', '00:40', '01:00', '01:20', '01:40', '02:00', '02:20', '02:40', '03:00', '03:20',
                    '03:40', '04:00', '04:20', '04:40', '05:00', '05:20', '05:40', '06:00', '06:20', '06:40', '07:00',
                    '07:20', '07:40', '08:00', '08:20', '08:40', '09:00', '09:20', '09:40', '10:00', '10:20', '10:40',
                    '11:00', '11:20', '11:40', '12:00', '12:20', '12:40', '13:00', '13:20', '13:40', '14:00', '14:20',
                    '14:40', '15:00', '15:20', '15:40', '16:00', '16:20', '16:40', '17:00', '17:20', '17:40', '18:00',
                    '18:20', '18:40', '19:00', '19:20', '19:40', '20:00', '20:20', '20:40', '21:00', '21:20', '21:40',
                    '22:00', '22:20', '22:40', '23:00', '23:20', 'Date']

file_name = "OCT_B_2018.csv"
df = pd.DataFrame(columns=csv_columns)
# name of the csv is in month+year+station_name    
dec = df.to_csv(file_name,index=False)

# device id 0 -> Firpeal Hostel Capacity -> 25 kW,Serial No -> I01161005922
# device id 1 -> Beauki Hostel, Capacity -> 15 kW, Serial No -> I01163009107
# device id 2 -> Chimair Hostel, Capacity -> 15 kW, Serial No -> I01161006486
# device id 3 -> Duven Hostel, Capacity -> 15 kW, Serial No -> I01161004187
# device id 4 -> Emiet Hostel Capacity -> 25 kW, Serial No -> I01161005921
# device id 5 -> No Working
# device id 6 -> Working but still unknown
# device id 7 -> Not Working
# device id 8 -> Not Working
# device id 9 -> Not Working
# device id 10 -> Aibaan Hostel, Capacity -> 25 Kw,Serial No -> I01161005931
# device id 11 -> 
device_id = device_ids[1]
count = 1
for dd in range(1,31):
    print("count: ",count)
    count+=1
    l = []
    dict_t_e=get_data(device_id, yyyy, mm, dd)
    x = datetime.datetime(yyyy,mm,dd)
    #print(dict_t_e)
    for d in dict_t_e.keys():
        l.append(dict_t_e[d])
    l.append(x)
    with open(file_name, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(l)
    fp.close()

count:  1
count:  2
count:  3
count:  4
count:  5
count:  6
count:  7
count:  8
count:  9
count:  10
count:  11
count:  12
count:  13
count:  14
count:  15
count:  16
count:  17
count:  18
count:  19
count:  20
count:  21
count:  22


ConnectionError: HTTPConnectionPool(host='14.215.130.185', port=20000): Max retries exceeded with url: /reportjson/exportReport.action?level=3&type=1&searchIds=32977&searchDate=2018-10-22 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002A4F0600198>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))